In [ ]:
import csv
import os
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import cv2
import numpy as np

class ObjectTracker:
    def __init__(self, model_path, video_folder, pixel_to_meter):
        # Load YOLOv8 model
        self.model = YOLO(model_path)

        # Initialize DeepSORT tracker
        self.tracker = DeepSort(max_age=3, max_cosine_distance=0.01)

        # Store previous positions and tracking data
        self.previous_positions = {}
        self.tracking_data = []
        self.pixel_to_meter = pixel_to_meter  # Conversion factor from pixels to meters

        # Get all video files in the folder
        self.video_files = [f for f in os.listdir(video_folder) if f.endswith('.MOV')]
        self.video_folder = video_folder

        if not self.video_files:
            raise ValueError("No .MOV files found in the specified video folder.")

    def calculate_velocity(self, track_id, current_position, fps):
        if track_id in self.previous_positions:
            previous_position = self.previous_positions[track_id]
            displacement = np.linalg.norm(current_position - previous_position)
            velocity = displacement * fps * self.pixel_to_meter
        else:
            velocity = 0
        self.previous_positions[track_id] = current_position
        return velocity

    def draw_bounding_box(self, frame, bbox, track_id, velocity):
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (int(bbox[0]), int(bbox[1])-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f'Vel: {velocity:.2f} m/s', (int(bbox[0]), int(bbox[1])-40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise Exception(f"Could not open video file: {video_path}")

        fps = cap.get(cv2.CAP_PROP_FPS)

        self.previous_positions.clear()
        self.tracking_data.clear()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = self.model(frame)
            detections = results[0].boxes.data

            dets = []
            for box in detections:
                x1, y1, x2, y2, conf, cls = box.cpu().numpy()
                width = x2 - x1
                height = y2 - y1
                dets.append([[x1, y1, width, height], conf])

            tracks = self.tracker.update_tracks(dets, frame=frame)

            for track in tracks:
                if track.is_confirmed():
                    track_id = track.track_id
                    bbox = track.to_tlbr()
                    x_center = (bbox[0] + bbox[2]) / 2
                    y_center = (bbox[1] + bbox[3]) / 2
                    current_position = np.array([x_center, y_center])

                    velocity = self.calculate_velocity(track_id, current_position, fps)
                    self.draw_bounding_box(frame, bbox, track_id, velocity)

                    self.tracking_data.append([cap.get(cv2.CAP_PROP_POS_FRAMES), track_id, x_center, y_center, velocity])

        cap.release()

    def export_tracking_data(self, csv_path):
        with open(csv_path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Frame', 'Object ID', 'X Center', 'Y Center', 'Velocity (m/s)'])
            csvwriter.writerows(self.tracking_data)

    def run(self):
        for video_file in self.video_files:
            video_path = os.path.join(self.video_folder, video_file)

            output_folder = os.path.join(self.video_folder, video_file.replace('.MOV', '_output'))
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            print(f"Processing video: {video_file}")
            self.process_video(video_path)
            self.export_tracking_data(os.path.join(output_folder, f'{video_file.replace(".MOV", "_tracking_data.csv")}'))

# Usage
pixel_to_meter = 0.01  # Example conversion factor
video_folder = r'C:\Users\MAY02\Desktop\FishTracking-\Test'  # Ensure this path is correct

tracker = ObjectTracker('bestYOLOV8x.pt', video_folder, pixel_to_meter)

try:
    tracker.run()
except Exception as e:
    print(f"An error occurred: {e}")


Processing video: MaMe-FeMos-FeMe-d4-5ng.MOV

0: 384x640 9 fishs, 63.2ms
Speed: 5.3ms preprocess, 63.2ms inference, 174.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 10.5ms
Speed: 3.0ms preprocess, 10.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 11.0ms
Speed: 3.5ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 10.0ms
Speed: 2.5ms preprocess, 10.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 10.0ms
Speed: 3.5ms preprocess, 10.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 10.0ms
Speed: 2.4ms preprocess, 10.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 fishs, 11.0ms
Speed: 3.5ms preprocess, 11.0ms inference, 1.0ms p

In [ ]:
706.0699559	364.4589487
706.6905643	361.974339


In [4]:
364.4589487-361.974339

2.4846097000000213

In [2]:
import numpy as np

In [5]:
np.sqrt(0.6206084000000374**2 + 2.4**2)


2.47894227164543